In [1]:
# Computer-Aided Civil and Infrastructure Engineering
# https://onlinelibrary.wiley.com/loi/14678667

idNo = '14678667'

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv
import re

In [3]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

global browser
global df
global row

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [4]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:

# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['and']
# searchTerm2 = ['of']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']


In [6]:
def getData(row,link):
#     browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
#     browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('h1',{'class':'citation__title'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title

    
    # FOR ABSTRACT SECTION
    
    absSection = parsed_html.find('section',{'class':'article-section article-section__abstract'})
    if absSection is not None:
        try:
            absSectionData = parsed_html.find('div',{'class':'article-section__content en main'}).find('p').text

            df.loc[row,'section0'] = absSection.find('h2').text
            df.loc[row,'sectionData0'] = parsed_html.find('div',{'class':'article-section__content en main'}).find('p').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

    # FOR ALL OTHER SECTIONS
        
    sections = parsed_html.find_all('section',{'class':'article-section__content'})
    print("No of sections: " + str(len(sections)))
    
    for s in range(len(sections)):
        if s is not None:  
            sectionNo = 'section' + str(s)
            sectionData = 'sectionData' + str(s)
            try:
                df.loc[row,sectionNo] = sections[s].find('h2').text
            except:
                df.loc[row,sectionNo] = "None"
        
            secData = sections[s].find_all('p')
            fullPara = ''
            for para in secData:
                fullPara += para.text.strip()
            df.loc[row,sectionData] = fullPara
        else:
            df.loc[row,sectionNo] = "None"             

In [7]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [14]:
baseUrl = 'https://onlinelibrary.wiley.com/action/doSearch?'
browser = webdriver.Safari()
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = baseUrl + "field1=Abstract&text1=%22" + item1 + "%22+AND+%22" + item2 + "%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub="
        print("base query: ",sq)
        query = sq
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Call webdriver         

        browser.get(query)
        respData = browser.page_source
        
        result = re.search('digitalData = (.*)</script>', respData)
        py_dic = json.loads(result.group(1))

        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'search':
                pageResults = py_dic[key]['resultCounts']['articlesChapters']

        print("pageResults: ",pageResults)
        if pageResults != 0:
            if pageResults > 20:
                page = pageResults // 20
                lastPage = pageResults % 20
                if lastPage > 0:
                    page = page + 1
            else:
                page = 1
            print("page: ",page)

            for p in range(2):        
    #         for p in range(page):
                print("In page: ",p)
                if p == 0:
                    query = sq
                else:
                    offset = str(p)
                    query = baseUrl + "Ppub=&field1=Abstract&field2=AllField&publication%5B%5D=14678667&text1=%22" + item1 + "%22+AND+%22" + item2 + "%22&text2=&pageSize=20&startPage="+offset
                    browser.get(query)
                    respData = browser.page_source

                print("search query: ",query)  
                parsed_html = BeautifulSoup(respData)
                aallParas = parsed_html.find_all('span',{'class':'hlFld-Title'})
                base = "https://onlinelibrary.wiley.com"
                for a in aallParas:
    #                 print(a.text)
                    doi = a.find('a').get('href')
                    link = base + doi
    #                 print(link)
                    print('Reading row: ',row)
                    df.loc[row,'searchTerm1'] = item1
                    df.loc[row,'searchTerm2'] = item2
                    getData(row,link)
                    row = row + 1
        else:
            print("No results found")
            
            
browser.quit()            

base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Buildings%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Sustainable+building%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Sustainable+buildings%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Environmentally+friendly+design%22&field2=AllField&text2=&

pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainability%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Sustainable+development%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Energy%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Water%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doS

pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Electricity%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Solar%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Wind+power%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Green+infrastructure%22&field2=AllField&text2=&publication%5B%5D=14678667&Ppub=
pageResults:  0
No results found
base query:  https://onlinelibrary.wiley.com/action/doSearch?field1=Abstract&text

pageResults:  0
No results found


In [18]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName


In [19]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName


In [20]:
df1 = df
df['journalName'] = 'wiley'
fileSave = 'kgRawData/wiley.csv'
df.to_csv(fileSave)

In [27]:
import pandas as pd
data = pd.read_csv('kgRawData/SD/sd-kg271089.csv')
data['presence'] = False

In [28]:
searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

<ipython-input-28-b50795caeeff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


In [35]:
uniqueValidEntries = list(set(validEntries))

In [36]:
uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''

In [44]:
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]

In [46]:
uniqueDf['url'][0]

'https://www.sciencedirect.com//science/article/pii/S0378778820300116'

In [47]:
uniqueDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM interface for full vs. semi-automated buil...,Abstract,BIM (building information model) enables infor...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Sustainability,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,An integrated BIM-based framework for the opti...,Abstract,Design choices with a unilateral focus on the ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BIM,Sustainability,https://www.sciencedirect.com//science/article...,An integrated BIM-based framework for the opti...,Abstract,Design choices with a unilateral focus on the ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM-based techno-economic assessment of energy...,Abstract,Buildings in hot humid climates are energy int...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,BIM interface for full vs. semi-automated buil...,Abstract,BIM (building information model) enables infor...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,Quantitative approach for evaluating the build...,Abstract,Architectural building design practic in hot c...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
uniqueDf.to_csv('kgRawData/SD/cleanSd-kg271089.csv')

In [50]:
len(uniqueDf)

39